In [1]:
import polars as pl

In [2]:
df = pl.read_csv('data/PD 2023 Wk 40 Input.csv')
df.head()

Name,Class 1,Class 2,Class 3,Class 4,Dropped Class 1,Dropped Class 2,Dropped Class 3
str,str,str,str,str,str,str,str
"""Camila Johnson…","""French""","""Science""","""English""","""Math""","""English""","""Science""",null
"""Emma Taylor""","""french""","""History""","""Geography""","""Science""","""Null""",null,null
"""Mia Johnson""","""Science""","""Geography""","""Physqcal Educa…","""English""","""Null""",null,null
"""Ella Young""","""Math""","""Sciwnce""","""English""","""Physical Educa…","""Physical Educa…",null,null
"""Mila Hall""","""Geography""","""Histpry""","""Physical Educa…","""English""","""Physical Educa…","""History""","""Geogrlphy"""


In [3]:
df.glimpse()

Rows: 250
Columns: 8
$ Name            <str> 'Camila Johnson', 'Emma Taylor', 'Mia Johnson', 'Ella Young', 'Mila Hall', 'Emma Lewis', 'Camila Wilson', 'Avery Parker', 'Emily Garcia', 'Scarlett Adams'
$ Class 1         <str> 'French', 'french', 'Science', 'Math', 'Geography', 'Math', 'History', 'Sciegce', 'Science', 'Geography'
$ Class 2         <str> 'Science', 'History', 'Geography', 'Sciwnce', 'Histpry', 'English', 'Math', 'Geography', 'Physbcal Education', 'Mkth'
$ Class 3         <str> 'English', 'Geography', 'Physqcal Education', 'English', 'Physical Education', 'French', 'Physical Education', 'Hiszory', 'Math', 'Physical Education'
$ Class 4         <str> 'Math', 'Science', 'English', 'Physical Education', 'English', 'History', 'English', 'French', 'Geograpsy', 'Science'
$ Dropped Class 1 <str> 'English', 'Null', 'Null', 'Physical Education', 'Physical Education', 'History', 'Null', 'French', 'Math', 'Sciance'
$ Dropped Class 2 <str> 'Science', None, None, None, 'History', 'Fuenc

In [4]:
class_df = (
    df.melt(
        id_vars='Name', 
        variable_name='Class Type', 
        value_name='Subject', 
        value_vars=df.columns[1:]
    )
    .with_columns(
        # English
        pl.when(
            pl.col('Subject').str.starts_with('En')
            |
            pl.col('Subject').str.ends_with('sh')  
        )
        .then(pl.lit('English'))
        # French
        .when(
            pl.col('Subject').str.starts_with('Fren') 
            |
            pl.col('Subject').str.ends_with('ch')   
        )
        .then(pl.lit('French'))
        # Geography
        .when(
            pl.col('Subject').str.starts_with('Ge') 
            |
            pl.col('Subject').str.ends_with('phy')   
        )
        .then(pl.lit('Geography'))
        # History
        .when(
            pl.col('Subject').str.starts_with('Hi') 
            |
            pl.col('Subject').str.ends_with('ry')   
        )
        .then(pl.lit('History'))
        # Math
        .when(
            pl.col('Subject').str.starts_with('M') 
            |
            pl.col('Subject').str.ends_with('th')   
        )
        .then(pl.lit('Math'))
        # Null
        .when(
            pl.col('Subject') == 'Null'
        )
        .then(pl.lit(None))
        # Physical Education
        .when(
            pl.col('Subject').str.starts_with('Physi') |
            pl.col('Subject').str.contains('sical') |
            pl.col('Subject').str.contains('Edu') 
        )
        .then(pl.lit('Physical Education'))
        # Science
        .when(
            pl.col('Subject').str.starts_with('Sc') |
            pl.col('Subject').str.contains('cie') |
            pl.col('Subject').str.ends_with('ce') 
        )
        .then(pl.lit('Science'))
        .otherwise(pl.col('Subject')).alias('Subject'),
        # a new column
        pl.when(
            pl.col('Class Type').str.starts_with('Dropped')
        )
        .then(pl.lit('Dropped'))
        .otherwise(pl.lit('Active'))
        .alias('Class Status')
    )
    .filter(pl.col('Subject').is_not_null())
)
class_df.head()

Name,Class Type,Subject,Class Status
str,str,str,str
"""Camila Johnson…","""Class 1""","""French""","""Active"""
"""Emma Taylor""","""Class 1""","""French""","""Active"""
"""Mia Johnson""","""Class 1""","""Science""","""Active"""
"""Ella Young""","""Class 1""","""Math""","""Active"""
"""Mila Hall""","""Class 1""","""Geography""","""Active"""


In [22]:
agg_df = (
    class_df
    .group_by('Subject')
    .agg(
        pl.col('Class Status').filter(pl.col('Class Status')=='Dropped').count().alias('Dropped'),
        pl.col('Class Status').filter(pl.col('Class Status')=='Active').count().alias('Active'),
        pl.count().alias('Total Enrolled')
    )
    .with_columns(
        (pl.col('Dropped') / pl.col('Total Enrolled')).round(3).alias('Dropped Rate')
    )
    .sort('Dropped Rate', descending=True)
)
agg_df

Subject,Dropped,Active,Total Enrolled,Dropped Rate
str,u32,u32,u32,f64
"""Math""",64,150,214,0.299
"""Geography""",65,154,219,0.297
"""Physical Educa…",59,143,202,0.292
"""History""",63,155,218,0.289
"""French""",55,141,196,0.281
"""Science""",47,129,176,0.267
"""English""",44,128,172,0.256
